발표 : 6월 17일 / 6월 19일

면담 : E-R 다이어그램, 발표 면담으로 진행

> PPT? A4용지? 해보고 의견을 들어보는 방향으로

1. 어떤 DB를 썼는지
2. E-R 다이어그램 그림
3. 테이블 어떻게 만드는지(**스키마 관련**)
4. 정규화를 할 수 있다면

## 1. 라이브러리 imports

In [7]:
import sqlite3
import pandas as pd

In [8]:
con = sqlite3.connect(':memory:')  ## 메모리에 접근
con = sqlite3.connect('./test.db')  ## DB 파일에 접근 또는 생성

sql_query = """SELECT name FROM sqlite_master
            WHERE type = 'table';"""
## 테이블인 것의 이름을 전부 가져옴

cur = con.cursor()  ## con을 조작하기 위한 인스턴스 생성
cur.execute(sql_query)  ## execute query
print(cur.fetchall())  ## send query result
con.execute("PRAGMA foreign_keys = 1")  ## foreign_keys activate. 인스턴스에서 지정하는 파라메터가 아님.

[]


## 2. 데이터 베이스 생성

In [9]:
table = """ CREATE TABLE GEEK (
            Email VARCHAR(255) NOT NULL,
            First_Name CHAR(25) NOT NULL,
            Last_Name CHAR(25),
            Score INT
        ); """
## 스키마 지정 VARCHAR : 가변문자, CHAR : 그냥 문자
cur.execute(table)
print("Table is Ready")  ## debuging

cur.execute("SELECT * FROM GEEK")
print(cur.fetchall())  ## showing all tuples

Table is Ready
[]


In [10]:
cur.execute(sql_query)  ## 테이블 네임 호출
print(cur.fetchall())

[('GEEK',)]


In [11]:
cur.execute("INSERT INTO GEEK VALUES (?,?,?,?);", ( 'Ihj@jbnu.ac.kr', 'GY', 'KIM', 10) )
## 테이블과 넣을 튜플
cur.execute("SELECT * FROM GEEK")  ## 전부가져옴
print (cur.fetchall())

cur.execute('DELETE FROM GEEK WHERE Score > 20' )  ## score가 20 이상인 튜플을 삭제
cur.execute("SELECT * FROM GEEK")
print (cur.fetchall())

[('Ihj@jbnu.ac.kr', 'GY', 'KIM', 10)]
[('Ihj@jbnu.ac.kr', 'GY', 'KIM', 10)]


> UNIQUE하게 두지 않았기 때문에 계속 시행하면 무한정 늘어남.

## 3. 판다스 라이브러리와의 연동

In [12]:
import pandas as pd
df = pd.read_csv('Python/ZP.csv')
print(df)

df.to_sql('test', con)  ## con의 test 테이블에 df를 삽입
cur.execute("SELECT * FROM test")
print(cur.fetchall())

          e-mail    first  last  score
0    g@naver.com   Minhee  Kang     50
1    jk@daum.net  Shinwoo   Kim     70
2  hg@jbnu.ac.kr    Heedo  Kwon     90
3   gg@naver.com    Junsu   Lee     59
4  jop@gmail.com   Jungsu   Lee     89
[(0, 'g@naver.com', 'Minhee', 'Kang', 50), (1, 'jk@daum.net', 'Shinwoo', 'Kim', 70), (2, 'hg@jbnu.ac.kr', 'Heedo', 'Kwon', 90), (3, 'gg@naver.com', 'Junsu', 'Lee', 59), (4, 'jop@gmail.com', 'Jungsu', 'Lee', 89)]


In [13]:
cur.execute("INSERT INTO test VALUES (?,?,?,?,?);", (6,'aq@daum.net','Hong','Lee', 40))
ag = 'INSERT INTO test VALUES (?,?,?,?,?);'
vals = [(11,'d@naver.com','Heesu','Kim', 50), (12,'aq@daum.net','Hong','Lee', 40)]
## 리스트로 여러 개의 튜플을 집어넣을 수 있음.
cur = con.cursor()  ## create instance
cur.executemany(ag, vals)  ## insert many tuples
cur.execute("SELECT * FROM TEST")
print(cur.fetchall())  ## debuging

[(0, 'g@naver.com', 'Minhee', 'Kang', 50), (1, 'jk@daum.net', 'Shinwoo', 'Kim', 70), (2, 'hg@jbnu.ac.kr', 'Heedo', 'Kwon', 90), (3, 'gg@naver.com', 'Junsu', 'Lee', 59), (4, 'jop@gmail.com', 'Jungsu', 'Lee', 89), (6, 'aq@daum.net', 'Hong', 'Lee', 40), (11, 'd@naver.com', 'Heesu', 'Kim', 50), (12, 'aq@daum.net', 'Hong', 'Lee', 40)]


`-` 테이블 조작 : 추가 삽입

In [14]:
cur.execute("DELETE FROM test WHERE score > 40")
cur.execute("SELECT * FROM test")
print(cur.fetchall())
cur.execute("UPDATE TEST SET SCORE = 60 WHERE SCORE = 40")
cur.execute("SELECT * FROM test")
print(cur.fetchall())

[(6, 'aq@daum.net', 'Hong', 'Lee', 40), (12, 'aq@daum.net', 'Hong', 'Lee', 40)]
[(6, 'aq@daum.net', 'Hong', 'Lee', 60), (12, 'aq@daum.net', 'Hong', 'Lee', 60)]


In [16]:
cur.execute(sql_query)  ## 스키마 마스터 이름을 호출
cur.fetchall()

[('GEEK',), ('test',)]

`-` 현재 테이블들을 확인

In [17]:
table_db = """ CREATE TABLE target_db (
            Email VARCHAR(255) PRIMARY KEY,
            First_Name CHAR(25) NOT NULL,
            Last_Name CHAR(25),
            Score INT CHECK(Score>10)
        ); """

cur.execute(table_db)  ## 새 테이블 생성
cur = con.cursor()
print(df.itertuples())
for row in df.itertuples():
    sql = "INSERT INTO target_db(Email, First_Name, Last_Name, Score) VALUES (?, ?, ?, ?)"
    cur.execute(sql, (row[1], row[2], row[3], row[4]))
con.commit()
#cur.execute('DELETE FROM target_db')
cur.execute('SELECT * FROM target_db')
print(cur.fetchall())

[('g@naver.com', 'Minhee', 'Kang', 50), ('jk@daum.net', 'Shinwoo', 'Kim', 70), ('hg@jbnu.ac.kr', 'Heedo', 'Kwon', 90), ('gg@naver.com', 'Junsu', 'Lee', 59), ('jop@gmail.com', 'Jungsu', 'Lee', 89)]


`-` 테이블 조작 : 스키마 지정 테이블 생성 및 입력

1. 스키마 지정 테이블 생성
2. 루프문을 이용하여 외부데이터 입력
3. 내용 확인

In [18]:
cur.execute("INSERT INTO target_db(Email, First_Name, Last_Name, Score) values ('G@NAVER.COM','kYUNGSU','LEE', 49)")
cur.execute('SELECT * FROM  target_db')
print(cur.fetchall())
cur.execute("INSERT INTO target_db(Email, First_Name, Last_Name, Score) values ('g@naver.com','kYUNGSU','LEE', 49)")

[('g@naver.com', 'Minhee', 'Kang', 50), ('jk@daum.net', 'Shinwoo', 'Kim', 70), ('hg@jbnu.ac.kr', 'Heedo', 'Kwon', 90), ('gg@naver.com', 'Junsu', 'Lee', 59), ('jop@gmail.com', 'Jungsu', 'Lee', 89), ('G@NAVER.COM', 'kYUNGSU', 'LEE', 49)]


IntegrityError: UNIQUE constraint failed: target_db.Email

> 유니크함이 깨짐. 이메일이 기본키이므로 같은 것을 넣으면 깨짐.

**기본키에서 똑같은 값을 넣으면 작동이 안되는지 확인해야**

`-` 기본키 + 제약조건. 뒤에는 필요없는 중복 내용

In [19]:
cur.execute('DROP table target_db')
table_db = """ CREATE TABLE target_db (
            ID INTEGER ,
            First_Name CHAR(25) NOT NULL,
            Last_Name CHAR(25),
            Score INT CHECK(Score>10) PRIMARY KEY NOT NULL UNIQUE
        ); """
cur.execute(table_db)

In [21]:
cur.execute(sql_query)
print(cur.fetchall())

[('GEEK',), ('test',), ('target_db',)]


In [23]:
rr = cur.execute("PRAGMA table_info('target_db')")
for r2 in rr :
    print(r2)

(0, 'ID', 'INTEGER', 0, None, 0)
(1, 'First_Name', 'CHAR(25)', 1, None, 0)
(2, 'Last_Name', 'CHAR(25)', 0, None, 0)
(3, 'Score', 'INT', 1, None, 1)


> 인덱스, 열1, 열2, 열3. 뒤의 세 값은 어떤 것을 의미하는지 메뉴얼을 찾아봐야 함.

`-` 기본키와 외래키, 스키마 전부 출력

In [24]:
rows = cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")
tables = [row[0] for row in rows]
print(tables)

def sql_identifier(s):
    return '"' + s.replace('"', '""') + '"'

for table in tables:
    print("table: " + table)
    rows = cur.execute("PRAGMA table_info({})".format(sql_identifier(table)))
    print(rows.fetchall())
    rows = cur.execute("PRAGMA foreign_key_list({})".format(sql_identifier(table)))
    print(rows.fetchall())


['GEEK', 'test', 'target_db']
table: GEEK
[(0, 'Email', 'VARCHAR(255)', 1, None, 0), (1, 'First_Name', 'CHAR(25)', 1, None, 0), (2, 'Last_Name', 'CHAR(25)', 0, None, 0), (3, 'Score', 'INT', 0, None, 0)]
[]
table: test
[(0, 'index', 'INTEGER', 0, None, 0), (1, 'e-mail', 'TEXT', 0, None, 0), (2, 'first', 'TEXT', 0, None, 0), (3, 'last', 'TEXT', 0, None, 0), (4, 'score', 'INTEGER', 0, None, 0)]
[]
table: target_db
[(0, 'ID', 'INTEGER', 0, None, 0), (1, 'First_Name', 'CHAR(25)', 1, None, 0), (2, 'Last_Name', 'CHAR(25)', 0, None, 0), (3, 'Score', 'INT', 1, None, 1)]
[]


In [25]:
con.close()  ## 데이터베이스와 연결 종료

In [29]:
conn = sqlite3.connect(':memory:')
conn = sqlite3.connect('./test1.db')  ## 새로운 DB test1을 만듦
conn.execute("PRAGMA foreign_keys = 1")

cur = conn.cursor()

cur = conn.cursor()
table_db = """ CREATE TABLE target_db (
            EMail CHAR(255),
            First_Name CHAR(25) NOT NULL,
            Last_Name CHAR(25) PRIMARY KEY,
            Score INT CHECK(Score>10) NOT NULL UNIQUE
        ); """
cur.execute(table_db)
tg = """   CREATE TABLE target (
            Email CHAR(255),
            First_Name CHAR(25) NOT NULL,
            Last_Name CHAR(25),
            Score INT CHECK(Score>10),
            FOREIGN KEY (Last_Name) REFERENCES target_db(Last_Name)
            ON DELETE SET NULL ON UPDATE SET NULL
        ); """
cur.execute(tg)

cur.execute(sql_query)
print(cur.fetchall())

[('target_db',), ('target',)]


`-` 외래키

In [30]:
ag = 'INSERT INTO target_db VALUES (?,?,?,?);'
vals = [('d@naver.com','Heesu','KIM', 50), ('Ihj@jbnu.ac.kr','Hong','LEE', 40)]

cur.executemany(ag, vals)  ## 값을 직접 삽입
cur.execute('SELECT * FROM target_db')
print(cur.fetchall())

ag = 'INSERT INTO target VALUES (?,?,?,?);'
vals = [('gg@naver.com','Heesu','KIM', 40), ('weg@jbnu.ac.kr','Dong','LEE', 20)]

cur.executemany(ag, vals)  ## 또 직접 삽입

cur.execute('SELECT * FROM target')
print(cur.fetchall())

[('d@naver.com', 'Heesu', 'KIM', 50), ('Ihj@jbnu.ac.kr', 'Hong', 'LEE', 40)]
[('gg@naver.com', 'Heesu', 'KIM', 40), ('weg@jbnu.ac.kr', 'Dong', 'LEE', 20)]


In [31]:
cur.execute("DELETE FROM target_db WHERE Last_Name='KIM'")
cur.execute('SELECT * FROM target_db')
print(cur.fetchall())

cur.execute("PRAGMA foreign_keys = ON")  ## 이건 con에 해야됨
cur.execute('SELECT * FROM target')
print(cur.fetchall())

[('Ihj@jbnu.ac.kr', 'Hong', 'LEE', 40)]
[('gg@naver.com', 'Heesu', None, 40), ('weg@jbnu.ac.kr', 'Dong', 'LEE', 20)]


In [32]:
tg = """ CREATE TABLE tmpp (
         Email CHAR(255),
         First_Name CHAR(25),
         Last_Name CHAR(25),
         Score INTEGER,
         FOREIGN KEY (Last_Name) REFERENCES target_db(Last_Name)
         ON DELETE CASCADE ON UPDATE CASCADE
     ); """
cur.execute(tg)

cur.execute('INSERT INTO tmpp SELECT * FROM target')
# cur.execute('SELECT * FROM tmpp').fetchall()
cur.execute("DROP TABLE target")
cur.execute("ALTER TABLE tmpp RENAME to target")

cur.execute("UPDATE target_db SET Last_Name = 'le' WHERE Last_Name = 'LEE'")
cur.execute("SELECT * FROM target").fetchall()

[('gg@naver.com', 'Heesu', None, 40), ('weg@jbnu.ac.kr', 'Dong', 'le', 20)]